In [3]:
import pandas as pd
import numpy as np

In [2]:
import os, glob

inputs = []

for each in glob.glob(os.path.join(os.getcwd(),'**/*.jsonl'), recursive=True):
    print(f'Name with path: {each} \nName without path: {os.path.basename(each)}')
    inputs.append(each)

Name with path: /Users/bindas/Documents/SEM6/IUM/Dane/users.jsonl 
Name without path: users.jsonl
Name with path: /Users/bindas/Documents/SEM6/IUM/Dane/deliveries.jsonl 
Name without path: deliveries.jsonl
Name with path: /Users/bindas/Documents/SEM6/IUM/Dane/sessions.jsonl 
Name without path: sessions.jsonl
Name with path: /Users/bindas/Documents/SEM6/IUM/Dane/products.jsonl 
Name without path: products.jsonl


In [17]:
users = pd.read_json(inputs[0], lines=True)
deliveries = pd.read_json(inputs[1], lines=True)
sessions = pd.read_json(inputs[2], lines=True)
products = pd.read_json(inputs[3], lines=True)

In [18]:
users.describe()

,user_id
count,200.000000
mean,201.500000
std,57.879185
min,102.000000
25%,151.750000
50%,201.500000
75%,251.250000
max,301.000000


In [20]:
users.sample(3)

,user_id,name,city,street
114,216,Kacper Markiel,Warszawa,ul. Szymanowskiego 950
125,227,Lidia Idzi,Konin,aleja Konarskiego 19/22
30,132,Krystian Piekara,Konin,pl. Waryńskiego 99/70


In [19]:
deliveries.describe()

,purchase_id,delivery_company
count,5297.000000,5045.000000
mean,22649.000000,498.261249
std,1529.256519,106.646521
min,20001.000000,360.000000
25%,21325.000000,360.000000
50%,22649.000000,516.000000
75%,23973.000000,620.000000
max,25297.000000,620.000000


In [21]:
deliveries.sample(3)

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
3659,23660,2021-02-21T01:44:09,2021-02-26T01:47:59,360.0
24,20025,2021-02-15T12:42:32,2021-02-19T12:45:19,620.0
1392,21393,2021-03-05T05:05:40,2021-03-07T05:09:30,360.0


In [22]:
sessions.describe()

,session_id,user_id,product_id,offered_discount,purchase_id
count,37539.000000,35657.000000,35696.000000,37539.000000,5297.000000
mean,105310.580729,199.224388,1128.044851,4.921815,22649.000000
std,3049.208276,57.835868,119.183838,7.074449,1529.256519
min,100001.000000,102.000000,1001.000000,0.000000,20001.000000
25%,102677.000000,150.000000,1035.000000,0.000000,21325.000000
50%,105326.000000,196.000000,1067.000000,0.000000,22649.000000
75%,107956.000000,252.000000,1278.000000,10.000000,23973.000000
max,110597.000000,301.000000,1319.000000,20.000000,25297.000000


In [23]:
sessions.sample(3)

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
31693,108966,2021-02-08 11:14:44,269.0,1067.0,BUY_PRODUCT,15,24470.0
20949,105916,2021-02-15 01:49:27,207.0,1044.0,VIEW_PRODUCT,0,NaN
115,100032,2021-02-27 22:19:18,102.0,1005.0,VIEW_PRODUCT,0,NaN


In [24]:
products.describe()

,product_id,price
count,319.000000,3.190000e+02
mean,1160.000000,5.370644e+06
std,92.231593,4.231655e+07
min,1001.000000,-2.450000e+02
25%,1080.500000,2.390000e+01
50%,1160.000000,4.499000e+01
75%,1239.500000,1.109900e+02
max,1319.000000,6.887800e+08


In [25]:
products.sample(3)

,product_id,product_name,category_path,price
167,1168,Angry Birds Space (PC),Gry i konsole;Gry komputerowe,24.99
84,1085,Dishonored (PC),Gry i konsole;Gry komputerowe,34.99
29,1030,LCD Iiyama B2280WSD,Komputery;Monitory;Monitory LCD,739.00
